In [212]:
import numpy as np

In [213]:
from music21 import *

# Preprocessing 


In [65]:
import glob

In [108]:
all_notes=[]
for i in glob.glob('MIDI Files_combined/*.mid'):
    file=converter.parse(i) #converts into a stream object
    print('parsing {}'.format(i))
    parse_notes=None
    parse_notes=file.flat.notes
    
    for e in parse_notes:
        if isinstance(e,note.Note):
            all_notes.append(str(e.pitch))
        elif isinstance(e,chord.Chord):
            all_notes.append('*'.join(str(j) for j in e.normalOrder))
        else:
            continue

parsing MIDI Files_combined\0fithos.mid
parsing MIDI Files_combined\8.mid
parsing MIDI Files_combined\ahead_on_our_way_piano.mid
parsing MIDI Files_combined\AT.mid
parsing MIDI Files_combined\AUX - A Perfect World (128Bpm).mid
parsing MIDI Files_combined\AUX - Anymore (136Bpm).mid
parsing MIDI Files_combined\AUX - Aspects (139Bpm).mid
parsing MIDI Files_combined\AUX - Best Feeling (138Bpm).mid
parsing MIDI Files_combined\AUX - Came Along Way (129Bpm).mid
parsing MIDI Files_combined\AUX - Can't Quit (135Bpm).mid
parsing MIDI Files_combined\AUX - Canary OST (130Bpm).mid
parsing MIDI Files_combined\AUX - Epidemic (124Bpm).mid
parsing MIDI Files_combined\AUX - Essential (140Bpm).mid
parsing MIDI Files_combined\AUX - Even Better (145Bpm).mid
parsing MIDI Files_combined\AUX - Formula (129Bpm).mid
parsing MIDI Files_combined\AUX - Good People (130Bpm).mid
parsing MIDI Files_combined\AUX - Heard Before (140Bpm).mid
parsing MIDI Files_combined\AUX - Hopeful (134Bpm).mid
parsing MIDI Files_combi

In [111]:
len(all_notes)


62584

In [113]:
all_notes[:10]

['4*9', 'E2', '4*9', '4*9', '4*9', '4*9', '4*9', '4*9', '4*9', '11*4']

In [116]:
import pickle
with open('all_notes.pickle', 'wb') as handle:
    pickle.dump(all_notes, handle)

In [214]:
with open('all_notes.pickle', 'rb') as handle:
    notes = pickle.load(handle)

In [215]:
uniq=np.unique(notes,return_counts=True)
len(uniq[1])

363

In [216]:
notes_to_index={value:key for key,value in enumerate(sorted(uniq[0]))}

In [217]:
index_to_notes={key:value for key,value in enumerate(sorted(uniq[0]))}

In [218]:
notes_to_index

{'0': 0,
 '0*1': 1,
 '0*1*5': 2,
 '0*1*6': 3,
 '0*2': 4,
 '0*2*3*7': 5,
 '0*2*4*7': 6,
 '0*2*5': 7,
 '0*2*6': 8,
 '0*2*7': 9,
 '0*3': 10,
 '0*3*5': 11,
 '0*3*5*8': 12,
 '0*3*6': 13,
 '0*3*6*8': 14,
 '0*3*6*9': 15,
 '0*3*7': 16,
 '0*4': 17,
 '0*4*5': 18,
 '0*4*6': 19,
 '0*4*7': 20,
 '0*5': 21,
 '0*5*6': 22,
 '0*6': 23,
 '1': 24,
 '1*2': 25,
 '1*2*4*6*8*10': 26,
 '1*2*6': 27,
 '1*2*6*8': 28,
 '1*3': 29,
 '1*3*5': 30,
 '1*3*5*8': 31,
 '1*3*6': 32,
 '1*3*7': 33,
 '1*3*8': 34,
 '1*4': 35,
 '1*4*6': 36,
 '1*4*6*9': 37,
 '1*4*7': 38,
 '1*4*7*10': 39,
 '1*4*7*9': 40,
 '1*4*8': 41,
 '1*5': 42,
 '1*5*8': 43,
 '1*5*9': 44,
 '1*6': 45,
 '1*7': 46,
 '10': 47,
 '10*0': 48,
 '10*0*2*5': 49,
 '10*0*3': 50,
 '10*0*4': 51,
 '10*0*5': 52,
 '10*1': 53,
 '10*1*3': 54,
 '10*1*3*5*6': 55,
 '10*1*3*6': 56,
 '10*1*4': 57,
 '10*1*4*6': 58,
 '10*1*5': 59,
 '10*11': 60,
 '10*11*3': 61,
 '10*11*3*5': 62,
 '10*2': 63,
 '10*2*3': 64,
 '10*2*4': 65,
 '10*2*5': 66,
 '10*3': 67,
 '11': 68,
 '11*0': 69,
 '11*0*4': 70,
 

In [272]:
SEQUENCE_LENGTH=100
model_input=[]
model_output=[]

In [273]:
for i in range(len(notes)-SEQUENCE_LENGTH):
    inp=notes[i:i+SEQUENCE_LENGTH]
    out=notes[i+SEQUENCE_LENGTH]
    model_input.append([notes_to_index[j] for j in inp])
    model_output.append(notes_to_index[out])
    

In [283]:
type(model_input)

numpy.ndarray

In [275]:
#model_input=np.array(model_input)
model_input=np.reshape(model_input,(len(model_input),SEQUENCE_LENGTH,1))
#model_input.shape

In [276]:
n_model_input = model_input/float(len(uniq[1]))

In [277]:
from keras.utils import np_utils

In [278]:
model_output=np_utils.to_categorical(model_output)

In [279]:
model_output.shape

(62484, 363)

# Model

In [226]:
from keras.layers import *
from keras.models import Sequential


In [227]:
model=Sequential()
model.add(LSTM(512,input_shape=(model_input.shape[1],model_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(128))
model.add(Dense(len(uniq[1]),activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100, 256)          787456    
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_2 (Dense)              (None, 363)               46827     
Total params: 2,084,075
Trainable params: 2,084,075
Non-trainable params: 0
_________________________________________________________________


In [171]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

W0812 21:11:04.328611 14104 deprecation_wrapper.py:119] From c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [172]:
from keras.callbacks import ModelCheckpoint

In [173]:
checkpoint=ModelCheckpoint('model_after_each_epoch.h5',monitor='loss',mode='min',period=1)

In [ ]:
#hist=model.fit(n_model_input,model_output,batch_size=64,epochs=100,callbacks=[checkpoint])

In [228]:
from keras.models import load_model


In [292]:
m=load_model('weights-improvement-60-1.1406-bigger.hdf5')

## Time to predict

In [285]:
sequence_length = 100
network_input = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    network_input.append([notes_to_index[char] for char in sequence_in])

In [287]:
type(network_input)

list

In [297]:
random_note=np.random.randint(0,len(notes)-1)
t_n=network_input[random_note]

pred_out=[]
for p in range(200):
    #t_n=np.array(t_n)
    pred_input=np.reshape(t_n,(1,len(t_n),1))
    pred_input = pred_input / float(len(uniq[1]))
    pp=m.predict(pred_input)
    prediction=np.argmax(pp)
    pred_out.append(index_to_notes[prediction])
    print(prediction)
    #pred.append(pp)
    
    t_n.append(prediction) #adding latest index
    t_n =t_n[1:len(t_n)]
    

299
21
21
21
21
21
21
21
345
21
21
21
21
21
21
21
331
21
357
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
345
345
21
21
21
21
21
21
21
21
21
104
345
345
21
21
21
345
104
21
21
21
21
21
21
345
344
21
21
21
21
21
21
21
17
311
17
345
345
104
21
21
21
21
313
311
21
104
21
21
345
311
299
21
344
21
333
260
21
357
345
67
327
21
327
338
21
327
327
260
327
260
357
67
21
327
21
327
327
327
327
345
327
327
357
357
357
357
357
21
21
21
21
21
21
327
357
357
357
357
21
21
357
357
21
357
357
17
292
357
357
357
104
21
357
357
21
21
104
17
17
21
357
331
357
299
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
348
299
21
21
21
331
21
21
290
17
104
21
21
104
21
104
21


In [298]:
pred_out

['B0',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 'F1',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 'E-5',
 '0*5',
 'G1',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 'F1',
 'F1',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '2*4*9',
 'F1',
 'F1',
 '0*5',
 '0*5',
 '0*5',
 'F1',
 '2*4*9',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 'F1',
 'F#6',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 '0*4',
 'C#6',
 '0*4',
 'F1',
 'F1',
 '2*4*9',
 '0*5',
 '0*5',
 '0*5',
 '0*5',
 'C1',
 'C#6',
 '0*5',
 '2*4*9',
 '0*5',
 '0*5',
 'F1',
 'C#6',
 'B0',
 '0*5',
 'F#6',
 '0*5',
 'E1',
 '8*9*1*4',
 '0*5',
 'G1',
 'F1',
 '10*3',
 'E-1',
 '0*5',
 'E-1',
 'E6',
 '0*5',
 'E-1',
 'E-1',
 '8*9*1*4',
 'E-1',
 '8*9*1*4',
 'G1',
 '10*3',
 '0*5',
 'E-1',
 '0*5',
 'E-1',
 'E-1',
 'E-1',
 'E-1',
 'F1',
 'E-1'

In [299]:
offset=0.0
final_notes=[]
for i in range(len(pred_out)):
    i=pred_out[i]
    if i.isdigit() or ('*' in i):
        chord_notes=i.split('*')
        notes=[]
        for c in chord_notes:
            new_note=note.Note(int(c))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        #notes=[note.Note(int(j) for j in chord_notes)]
        
        c=chord.Chord(notes)
        c.offset=offset
        final_notes.append(c)
    else:
        n=note.Note(i)
        n.offset = offset
        n.storedInstrument = instrument.Piano()
        final_notes.append(n)
    offset+=0.5


In [300]:
midi_stream = stream.Stream(final_notes)
midi_stream.write('midi', fp='test_output1.mid')

'test_output1.mid'

In [301]:
midi_stream.show('midi')